**Utilities**

In [1]:
from anytree import Node, Resolver, RenderTree, PreOrderIter, PostOrderIter
from enum import Enum, auto
from collections import deque

"""
Node's hidden properties:
    parent, children, siblings, ancestors, descendants
    root, leaves, is_root, is_leaf
    height, depth, path, iter_path_reverse (iterate **up** the tree from here)
I'll also give it a kwarg 'size': int.

PostOrderIter lists leaves first. Good for calculating directory size efficiently.
    Use filter_=lambda n: n.is_leaf
"""

def read_line_s(file):
    """
    'Safe' version of TextIO.readline(). This one uses exceptions gracefully and returns the line without EOL characters. For this app, it splits the line into components the app will need.
    """
    line = f.readline()
    if line == '':
        raise EOFError
    return line.rstrip().split(' ')


class Mode(Enum):
    """
    Enumerated state of the state machine implemented in the main app
    """
    CMD = auto()
    RESP = auto()


class FileStructure:
    """
    A stateful app interface to the stateless "anytree" API which implements a tree structure for the "file system".
    """

    _root_name = '__r__'  # anytree seems happier overall when I use a named root node

    def __init__(self):
        self._root = Node(self._root_name, size=0, is_dir=True)
        self._r = Resolver('name')
        self._pwd = self._root

    def _clean(self, name: str) -> str:
        return self._root_name if name in ['', '/'] else name

    def _disp(self, name: str) -> str:
        return name.replace(self._root_name, '')

    @staticmethod
    def __incr_dir_sizes(file: Node):
        # I chose to implement this as an incremental, internalized method because the app already defines an incremental interface to the file system. (Its methods are only add() and cd(). The class already holds a lot of state information to provide the incremental interface, so why not have it statefully hold directory sizes and update them with each added file? A stateless interface would be nicer, but it doesn't make sense for this app.
        for a in file.ancestors:
            a.size += file.size
        return

    def render(self, maxlevel: any=None) -> str:
        _repr = deque()
        for pre, _, node in RenderTree(self._root, maxlevel=maxlevel):
            _repr.append(f'{pre}{node.name} {node.size}')
        _repr.appendleft('/' + self._disp(_repr.popleft()))
        return '\n'.join(_repr)

    def add(self, name: str, is_dir: bool, f_size: int=0):
        # Override size for directories so __incr_dir_size() works correctly
        _i = Node(self._clean(name), parent=self._pwd, size=f_size if not is_dir else 0, is_dir=is_dir)
        if not is_dir:
            self.__incr_dir_sizes(_i)

    def cd(self, name: str):
        # NB: In the input file, "cd [dir]" is always relative to pwd, except for the special case "cd /"
        if self._clean(name) == self._root_name:
            self._pwd = self._root
        else:
            self._pwd = self._r.get(self._pwd, name)

    @staticmethod
    def __resolve_name(item: Node) -> str:
        name = item.name
        while True:
            p = item.parent
            if p is None:
                return name
            else:
                name = p.name + '/' + name
                item = p

    @property
    def pwd(self) -> (str, int):
        return self._disp(self.__resolve_name(self._pwd)), self._pwd.size

    def list_all_dirs(self, size_limit: int=-1) -> list[tuple[str, int]]:
        _d = deque()
        if size_limit < 0:
            for i in PreOrderIter(self._root, filter_=lambda _: _.is_dir):
                _d.append((self._disp(self.__resolve_name(i)), i.size))
        else:
            for i in PreOrderIter(self._root, filter_=lambda _: _.is_dir and _.size <= size_limit):
                _d.append((self._disp(self.__resolve_name(i)), i.size))
        return list(_d)

# Test the class
test = FileStructure()
test.add('one', is_dir=True)
test.add('one', is_dir=False, f_size = 33)
test.cd('one')
test.add('two', is_dir=True)
test.cd('two')
test.cd('.')
test.add('three', is_dir=False, f_size=21)
test.add('two', is_dir=False, f_size=22)
print(f'pwd: {test.pwd}')
test.cd('..')
test.cd('/')
test.cd('..')
print('Test tree:\n' + test.render())
print(f'Directories: {test.list_all_dirs()}')

pwd: ('/one/two', 43)
Test tree:
/ 76
├── one 43
│   └── two 43
│       ├── three 21
│       └── two 22
└── one 33
Directories: [('', 76), ('/one', 43), ('/one/two', 43)]


**Part 1:**

In [2]:
# Build the file tree
file_tree = FileStructure()
mode = Mode.CMD
with open('../inputs/day7-input') as f:
    # State machine (flat design)
    while True:

        try:
            pieces = read_line_s(f)
        except EOFError:
            break

        print(f'{file_tree.pwd[0]} → ' + ' '.join(pieces))  # For debugging

        if mode == Mode.RESP:
            # Response mode
            if pieces[0] == 'dir':
                file_tree.add(pieces[1], is_dir=True)
                continue
            elif pieces[0].isdigit():
                file_tree.add(pieces[1], is_dir=False, f_size=int(pieces[0]))
                continue
            else:
                mode = Mode.CMD

        # Command mode
        if pieces[0] == '$':
            if pieces[1] == 'cd':
                file_tree.cd(pieces[2])
            elif pieces[1] == 'ls':
                mode = Mode.RESP  # Switch to Response mode for next line
            else:
                raise ValueError(f'{pieces[1]} is not a valid input command.')

tree_disp = file_tree.render()
print('\nDirectory tree:\n' + tree_disp)  # For debugging

#  Sum directories with size <= 1e6
my_small_dir_sum = sum([s for _, s in file_tree.list_all_dirs() if s <= 100000])
class_small_dir_sum = sum([s for _, s in file_tree.list_all_dirs(100000)])
print(f'Part 1: Class dirs <= 1e6 total: {class_small_dir_sum} | My dirs <= 1e6 total: {my_small_dir_sum} | Equal? {class_small_dir_sum == my_small_dir_sum}')


 → $ cd /
 → $ ls
 → dir fwbjchs
 → dir hmnpr
 → dir jtrbrcjl
 → dir lcgv
 → dir ldqc
 → dir vrvl
 → $ cd fwbjchs
/fwbjchs → $ ls
/fwbjchs → 154619 wqdlv.mdw
/fwbjchs → 21648 wvbnz
/fwbjchs → $ cd ..
 → $ cd hmnpr
/hmnpr → $ ls
/hmnpr → 178623 rftqqsrp.bfm
/hmnpr → $ cd ..
 → $ cd jtrbrcjl
/jtrbrcjl → $ ls
/jtrbrcjl → dir nmbfwc
/jtrbrcjl → dir whqb
/jtrbrcjl → $ cd nmbfwc
/jtrbrcjl/nmbfwc → $ ls
/jtrbrcjl/nmbfwc → 242645 lcgv
/jtrbrcjl/nmbfwc → 256365 wdzw.drg
/jtrbrcjl/nmbfwc → $ cd ..
/jtrbrcjl → $ cd whqb
/jtrbrcjl/whqb → $ ls
/jtrbrcjl/whqb → 161522 mrqgpv.gsm
/jtrbrcjl/whqb → 48062 vpsgcl.gfh
/jtrbrcjl/whqb → $ cd ..
/jtrbrcjl → $ cd ..
 → $ cd lcgv
/lcgv → $ ls
/lcgv → dir cthtlwds
/lcgv → dir grldv
/lcgv → dir lnztfr
/lcgv → dir vwhf
/lcgv → dir znmzg
/lcgv → $ cd cthtlwds
/lcgv/cthtlwds → $ ls
/lcgv/cthtlwds → dir dghvw
/lcgv/cthtlwds → dir tfwgg
/lcgv/cthtlwds → $ cd dghvw
/lcgv/cthtlwds/dghvw → $ ls
/lcgv/cthtlwds/dghvw → 107090 jmj.lzh
/lcgv/cthtlwds/dghvw → $ cd ..
/lcgv/c

**Part 2**

In [8]:
min_dir_size = 8381165

# I'm just gonna do this with the class I've built, rather than go mucking around with anytree again.
dirs = file_tree.list_all_dirs()
dirs_sorted = sorted(dirs, key=lambda _: _[1], reverse=True)
print('Directories:\n' + '\n'.join([f'{i[1]} {i[0]}' for i in dirs_sorted]))

Directories:
42080344 
34474241 /ldqc
26053496 /ldqc/jclb
19611481 /ldqc/jclb/pbb
13431333 /ldqc/jclb/pbb/zmflq
8121467 /ldqc/jclb/pbb/zmflq/lcgv
6180148 /ldqc/jclb/pbb/wqdlv
6033578 /lcgv
4548392 /ldqc/jclb/vpfggv
4243869 /ldqc/jclb/pbb/zmflq/wqdlv
3543702 /lcgv/vwhf
3493129 /ldqc/jclb/pbb/zmflq/lcgv/rfzrwc
3143244 /ldqc/jclb/pbb/zmflq/wqdlv/jrqsqrv
2224309 /ldqc/vhdgcsw
2103226 /ldqc/jclb/pbb/wqdlv/wqdlv
2086088 /ldqc/jclb/pbb/zmflq/lcgv/mzzpfnr
2050467 /ldqc/jclb/vpfggv/tpmnhdc
1787197 /ldqc/jclb/pbb/zmflq/wqdlv/jrqsqrv/zmflq
1682423 /ldqc/jclb/vpfggv/vvp
1573234 /ldqc/jclb/pbb/wqdlv/tgmqnq
1379420 /lcgv/cthtlwds
1357349 /lcgv/vwhf/bgtgqzz
1343597 /ldqc/gqb
1280789 /ldqc/jclb/pbb/zmflq/lcgv/dzz
1272330 /lcgv/cthtlwds/tfwgg
1223639 /lcgv/vwhf/wszvqd
1195283 /ldqc/rmljszcj
1164649 /ldqc/jgbsw
1101970 /ldqc/jclb/pbb/zmflq/lcgv/rfzrwc/bnsgzwhn
1086056 /ldqc/jclb/pbb/wqdlv/tgmqnq/lcgv
1075019 /ldqc/jclb/pbb/wqdlv/hrghbhj
1046537 /ldqc/jclb/pbb/wqdlv/wqdlv/ppsm
1039812 /ldqc/jclb/vpfggv/t